# Ranking
All Stage 2 work includes predicting individual features and scoring them.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
from sklearn import preprocessing
import geopandas as gpd
import folium

### Income

In [ ]:
df = pd.read_csv("../data/raw/datasource-AU_Govt_ABS-UoM_AURIN_DB_3_abs_data_by_region_income_asgs_sa2_2011_2019.csv")
SA2 = pd.read_csv("../data/curated/postcode_to_sa2.csv")

In [ ]:
df = df[['FID', 'sa2_maincode_2016', 'geometry', 'sa2_name_2016', 'yr','estm_prsnl_incme_yr_end_30_jne_tl_erns_excl_gvrnmt_pns_alwncs_n']]
df = df.rename({'estm_prsnl_incme_yr_end_30_jne_tl_erns_excl_gvrnmt_pns_alwncs_n': 'income'}, axis=1) 
df['income'].fillna(0, inplace=True)

In [ ]:
df_avg = df\
        .groupby('sa2_maincode_2016') \
        .agg(
            {
                'income':'mean',
            }
        ) 
df_avg.head()

In [ ]:
year = list(set(df.yr))
year.sort()

In [ ]:
year

In [ ]:
sa2_code = list(set(SA2.SA2_MAINCODE_2016))

In [ ]:
# Using the average of historical data growth rates to predict growth rates over the next three years

income_rate = {}
for code in sa2_code:
    
    df_temp = df[df['sa2_maincode_2016'] == code]
    num = 0
    rate = 0
    year = list(set(df_temp.yr))
    year.sort()
    
    # Skip this SA2 area if less or eaqul than one year of data exists and all other years is missing
    if(len(year) <= 1):
        next
    
    # Calculating the average growth rate
    for i in range(len(year)-1):
        curr_ = df_temp.loc[df_temp['yr'] == year[i],'income'].values[0] # Current year income
        next_ = df_temp.loc[df_temp['yr'] == year[i+1],'income'].values[0] # Next year income
        
        #Skip this year if this year and next year data is missing
        if(curr_ != 0 and next_ != 0):
            num += 1
            rate += (next_ - curr_)/curr_
            avg = rate/num
    income_rate[code] = round(avg,5)

In [ ]:
df_income = pd.DataFrame(income_rate.items(), columns=['SA2_2016', 'income_rate'])

In [ ]:
df_income

### Number of Bedrooms (census data)

2021

In [ ]:
df = pd.read_csv("../data/raw/2021_GCP_SA2_for_VIC_short-header/2021 Census GCP Statistical Area 2 for VIC/2021Census_G41_VIC_SA2.csv")

In [ ]:
df = df[['SA2_CODE_2021', 'Total_NofB_0_i_b', 'Total_NofB_1', 'Total_NofB_2', 'Total_NofB_3',
        'Total_NofB_4', 'Total_NofB_5','Total_NofB_6_or_m']]

In [ ]:
df

In [ ]:
beds_2021 = {}
for code in df.SA2_CODE_2021:
    df_temp = df[df['SA2_CODE_2021'] == code]
    
    Nob = 0 # number of beds
    Nop = 0 # number of property
    
    
    for i in range(len(df.columns)-1):
        curr_ = df_temp.iloc[:,i+1].values[0] # skip first SA2 code col
        Nop += curr_
        Nob += curr_ * i # number of property * number of beds for this col
        
    avg_beds = Nob/Nop
    beds_2021[code] = round(avg_beds,5)  

In [ ]:
df_bed_2021 = pd.DataFrame(beds_2021.items(), columns=['SA2_2021', 'avg_beds_2021'])
df_bed_2021.to_csv('../data/curated/df_bed_2021.csv',index=False)

2016

In [ ]:
df = pd.read_csv("../data/raw/2016_GCP_SA2_for_VIC_short-header/2016 Census GCP Statistical Area 2 for VIC/2016Census_G38_VIC_SA2.csv")

In [ ]:
df

In [ ]:
df = df[['SA2_MAINCODE_2016', 'Total_NofB_0_i_b', 'Total_NofB_1', 'Total_NofB_2', 'Total_NofB_3',
        'Total_NofB_4', 'Total_NofB_5','Total_NofB_6_or_m']]

In [ ]:
beds_2016 = {}
for code in df.SA2_MAINCODE_2016:
    df_temp = df[df['SA2_MAINCODE_2016'] == code]
    
    Nob = 0 # number of beds
    Nop = 0 # number of property
    
    for i in range(len(df.columns)-1):
        curr_ = df_temp.iloc[:,i+1].values[0]
        Nop += curr_
        Nob += curr_ * i
        
    avg_beds = Nob/Nop
    beds_2016[code] = round(avg_beds,5)  

In [ ]:
df_bed_2016 = pd.DataFrame(beds_2016.items(), columns=['SA2_2016', 'avg_beds_2016'])
df_bed_2016.to_csv('../data/curated/df_bed_2016.csv',index=False)

2011

In [ ]:
df = pd.read_csv("../data/raw/2011_BCP_SA2_for_VIC_short-header/2011 Census BCP Statistical Areas Level 2 for VIC/VIC/2011Census_B36_VIC_SA2_short.csv")

In [ ]:
df

In [ ]:
df = df[['region_id', 'Total_NofB_0_i_b', 'Total_NofB_1', 'Total_NofB_2', 'Total_NofB_3',
        'Total_NofB_4', 'Total_NofB_5','Total_NofB_6_or_m']]

In [ ]:
beds_2011 = {}
for code in df.region_id:
    df_temp = df[df['region_id'] == code]
    
    Nob = 0 # number of beds
    Nop = 0 # number of property
    
    for i in range(len(df.columns)-1):
        curr_ = df_temp.iloc[:,i+1].values[0]
        Nop += curr_
        Nob += curr_ * i
        
    avg_beds = Nob/Nop
    beds_2011[code] = round(avg_beds,5)  

In [ ]:
df_bed_2011 = pd.DataFrame(beds_2011.items(), columns=['SA2_2016', 'avg_beds_2011'])

In [ ]:
# merge all 3 years data
sa2 = SA2[['postcode','SA2_MAINCODE_2016','SA2_CODE_2021']]
sa2 = sa2.rename({'SA2_MAINCODE_2016': 'SA2_2016', 'SA2_CODE_2021': 'SA2_2021'}, axis=1) 
df_b_final = sa2.merge(df_bed_2021, on = 'SA2_2021')
df_b_final = df_b_final.merge(df_bed_2016, on = 'SA2_2016')
df_b_final = df_b_final.merge(df_bed_2011, on = 'SA2_2016')

In [ ]:
# Calculating the average growth rate
df_b_final['bed_rate'] = 0
for postcode in df_b_final.postcode:
    b2011 = df_b_final.loc[df_b_final['postcode'] == postcode, 'avg_beds_2011'].values[0]
    b2016 = df_b_final.loc[df_b_final['postcode'] == postcode, 'avg_beds_2016'].values[0]
    b2021 = df_b_final.loc[df_b_final['postcode'] == postcode, 'avg_beds_2021'].values[0]
    
    bed_rate = ((b2016 - b2011) + (b2021 - b2016))/2
    
    df_b_final.loc[df_b_final['postcode'] == postcode, 'bed_rate'] = bed_rate
    
df_b_final = df_b_final.drop(['avg_beds_2021','avg_beds_2016','avg_beds_2011'],axis=1)

In [ ]:
df_b_final

### Age and Population

Make a dic to store SA2 that have multiple postcode with the num of property in this postcode area

In [ ]:
domain = pd.read_csv("../data/curated/domain_final.csv")

In [ ]:
domain_sa2 = domain[['postcode','SA2_CODE21']]
domain_sa2 = domain_sa2.drop_duplicates()
domain_sa2 = domain_sa2.groupby('SA2_CODE21').count()
mul_sa2 = list(domain_sa2[domain_sa2['postcode'] > 1].index)

In [ ]:
mul_l = {}
for sa2 in mul_sa2:
    df_temp = domain[domain['SA2_CODE21'] == sa2].groupby('postcode').count()
    mul_l[sa2] = {}
    for postcode in list(df_temp.index):
        mul_l[sa2][postcode] = df_temp.loc[[postcode]]['name'].values[0]
    

Hendel all age and population data

In [ ]:
pp = pd.read_csv("../data/curated/df_p.csv")

In [ ]:
pp_2022 = pp[pp['YEAR'] == 2022]
pp_2025 = pp[pp['YEAR'] == 2025]
pp_2022 = pp_2022.rename({'SA2_CODE': 'SA2_2016'}, axis=1) 
pp_2025 = pp_2025.rename({'SA2_CODE': 'SA2_2016'}, axis=1) 

In [ ]:
df_pp_2022 = df_b_final.merge(pp_2022, on = 'SA2_2016')
df_pp_2022 = df_pp_2022.drop(['SA2_2016','Unnamed: 0','YEAR','SA2_NAME','SEX','rate','age_max','bed_rate'],axis=1)
df_pp_2025 = df_b_final.merge(pp_2025, on = 'SA2_2016')
df_pp_2025 = df_pp_2025.drop(['SA2_2016','Unnamed: 0','YEAR','SA2_NAME','SEX','rate','age_max','bed_rate'],axis=1)

In [ ]:
l_change = list(df_pp_2022.columns)[2:]

In [ ]:
for i in range(len(df_pp_2022)):
    sa2 = df_pp_2022.iloc[i,:].SA2_2021
    
    # if this post in the dictionary we create in first step we split SA2 data into postcode area
    if sa2 in mul_l.keys():
        postcode = df_pp_2022.iloc[i,:].postcode
        if postcode in mul_l[sa2].keys():
            for index in l_change:
                origin = df_pp_2022.loc[(df_pp_2022['postcode'] == postcode)&(df_pp_2022['SA2_2021'] == sa2), index].values[0]
                rate = mul_l[sa2][postcode] /  sum(list(mul_l[sa2].values()))
                new = origin * rate
                df_pp_2022.loc[(df_pp_2022['postcode'] == postcode)&(df_pp_2022['SA2_2021'] == sa2), index] = round(new,5)

In [ ]:
# After spliting data, calculating the mean value
df_pp_2022['total_age'] = 0
df_age_2022 = df_pp_2022.iloc[:,2:20]
for i in range(len(df_age_2022.columns)):
    
    # get low age value in every col
    low = int(re.findall(r'[0-9]+', df_age_2022.columns[i])[0])
    
    # avg age for this col is low value + 2,
    avg = (low + 2) 
    
    # calculating total_age in this postcode area = avg age for this col * num of people in this col
    df_pp_2022['total_age'] = df_pp_2022['total_age'] + avg * df_age_2022.iloc[:,i]

# total_age/total_population in this postcode area
df_pp_2022['avg_age'] = df_pp_2022['total_age']/df_pp_2022['Total']

In [ ]:
df_pp_2022

In [ ]:
# same as 2022 data
l_change = list(df_pp_2025.columns)[2:]

In [ ]:
for i in range(len(df_pp_2025)):
    sa2 = df_pp_2025.iloc[i,:].SA2_2021
    if sa2 in mul_l.keys():
        postcode = df_pp_2025.iloc[i,:].postcode
        if postcode in mul_l[sa2].keys():
            for index in l_change:
                origin = df_pp_2025.loc[(df_pp_2025['postcode'] == postcode)&(df_pp_2025['SA2_2021'] == sa2), index].values[0]
                rate = mul_l[sa2][postcode] / sum(list(mul_l[sa2].values()))
                new = origin * rate
                df_pp_2025.loc[(df_pp_2025['postcode'] == postcode)&(df_pp_2025['SA2_2021'] == sa2), index] = round(new,5)

In [ ]:
df_pp_2025['total_age'] = 0
df_age_2025 = df_pp_2025.iloc[:,2:20]
for i in range(len(df_age_2025.columns)):
    low = int(re.findall(r'[0-9]+', df_age_2025.columns[i])[0])
    avg = (low + 2) 
    df_pp_2025['total_age'] = df_pp_2025['total_age'] + avg * df_age_2025.iloc[:,i]
df_pp_2025['avg_age'] = df_pp_2025['total_age']/df_pp_2025['Total']

In [ ]:
df_pp_2025

In [ ]:
for i in range(len(df_pp_2025)):
    postcode = df_pp_2025.iloc[i,:].postcode
    p2025 = df_pp_2025.loc[df_pp_2025['postcode'] == postcode, 'Total'].values[0]
    p2022 = df_pp_2022.loc[df_pp_2022['postcode'] == postcode, 'Total'].values[0]
    a2025 = df_pp_2025.loc[df_pp_2025['postcode'] == postcode, 'avg_age'].values[0]
    a2022 = df_pp_2022.loc[df_pp_2022['postcode'] == postcode, 'avg_age'].values[0]
    df_pp_2025.loc[df_pp_2025['postcode'] == postcode, 'p_rate'] = round((p2025-p2022)/p2022,5)
    df_pp_2025.loc[df_pp_2025['postcode'] == postcode, 'a_rate'] = round((a2025-a2022)/a2022,5)


In [ ]:
df_pp_final = df_pp_2025[['postcode','SA2_2021','p_rate','a_rate']]

In [ ]:
df_pp_final 

<h1>Final Aggregatation</h1>

In [ ]:
df_final = df_b_final.merge(df_income, on = 'SA2_2016')
df_final = df_final.merge(df_pp_final, on = 'postcode')
df_final = df_final.drop('SA2_2021_y',axis=1)
df_final = df_final.rename({'SA2_2021_x':'SA2_2021'}, axis=1)

In [ ]:
df_final

<h1>Ranking</h1>

In [ ]:
# Normalization
x = df_final[['bed_rate', 'income_rate','p_rate','a_rate']].values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_final1 = pd.DataFrame(x_scaled, columns= ['bed_rate', 'income_rate','p_rate','a_rate'] )

In [ ]:
df_final['bed_rate'] = df_final1['bed_rate']
df_final['income_rate'] = df_final1['income_rate']
df_final['p_rate'] = df_final1['p_rate']
df_final['a_rate'] = df_final1['a_rate']

In [ ]:
df_final['rank'] = 0

# score based on rank of feature important in stage 1
df_final['rank'] += 3 * df_final.bed_rate + 2 * df_final.income_rate + 4 * df_final.a_rate + 1 * df_final.p_rate

#df_final.income_rate.max()

In [ ]:
domain = pd.read_csv('../data/curated/domain_final.csv')

In [ ]:
for postcode in df_final['postcode']:
    if postcode not in domain['postcode'].unique():
        index = df_final[ df_final['postcode'] == postcode ].index
        df_final.drop(index , inplace=True)

In [ ]:
top_10 = df_final.sort_values('rank',ascending=False).head(11)
index = df_final[ df_final['postcode'] == 3683 ].index
top_10.drop(index , inplace=True)

In [ ]:
top_10

In [ ]:
top_10
top_10.to_csv('../data/curated/growth_rate_final.csv',index=False)

<h1>Visualisation</h1>

In [ ]:
gdf = gpd.read_file("../data/raw/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")
gdf = gdf[gdf['STE_NAME21'] == 'Victoria']
gdf['geometry'] = gdf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf['SA2_CODE21'] = pd.to_numeric(gdf['SA2_CODE21'])
geoJSON = gdf[['SA2_CODE21', 'geometry']].drop_duplicates('SA2_CODE21').to_json()

In [ ]:
_map = folium.Map(location=[-37, 144], tiles="Stamen Terrain", zoom_start=7)


_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))

In [ ]:
postcode = pd.read_csv('../data/raw/Australian-Postcode-Data-master/au_postcodes.csv')
postcode = postcode.drop_duplicates(subset=['postcode'])
top_10 = top_10.merge(postcode[['postcode','place_name','latitude','longitude']], on = 'postcode')

In [ ]:
for i in range(len(top_10)):
    _map.add_child(
            folium.Marker(location=list(top_10[['latitude','longitude']].values[i]),
            popup= list(top_10[['place_name']].values[i])[0])
        )
_map

In [ ]:
top_10